# File Cleaning

This notebook cleans up any duplicate files uploaded to Google Drive. This notebook should be executed after the `data_upload.ipynb` Notebook. We noticed that of the txt files uploaded, the files ending with "2.txt" are duplicates and should be removed before performing our analysis. 

### Code
We do not use the PyDrive package for this activity due to issues we found. Instead, we will start by installing the following packages:

In [ ]:
# Install necessary packages
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [24]:
# Import packages
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

Here we create the function to authenticate the Google Drive API. Calling this function will prompt a similar authentication process to the one in the `data_upload.ipynb` Notebook.

In [30]:
# Authenticate Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
def authenticate_drive_api():
    creds = None
    # Check if token.pickle exists for stored credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If no valid credentials, authenticate using credentials.json
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('drive', 'v3', credentials=creds)

This function changes the "[trashed](https://developers.google.com/drive/api/guides/delete)" property of the file using the file ID.

In [32]:
def trash_file(service, file_id):
    try:
        # Update the file's 'trashed' property
        service.files().update(
            fileId=file_id,
            body={"trashed": True}
        ).execute()
        print(f"File {file_id} moved to trash.")
    except Exception as e:
        print(f"An error occurred: {e}")

This next function searches for the files with file names matching a pattern within a folder and then calls the `trash_file` function.

In [36]:
def find_and_trash_files(folder_id, pattern):
    # Query to find files in the specified folder
    query = f"'{folder_id}' in parents and trashed=false"

    # List files in the folder
    page_token = None
    while True:
        response = service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()

        for file in response.get('files', []):
            if pattern in file['name']:
                print(f"Trashing file: {file['name']} (ID: {file['id']})")
                # Move file to trash
                service.files().update(
                    fileId=file['id'],
                    body={"trashed": True}
                ).execute()

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

First, call the `authenticate_drive_api` function to grant access to the Google Drive account.

In [31]:
# Authenticate and create the API service
service = authenticate_drive_api()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=389849867563-4uggnm57nqe52156v32gj1lkosoqpoem.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54709%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=AHuwSrIU0ClxbMe5MEB0Y2vrit6tAF&access_type=offline


Next, define the folder ID for the folder of interest and the pattern to look for in the files to remove. The folder ID can be found as the string after the "folders/" part of the URL for the Google Drive folder.

Finally, we call the `find_and_trash_files` function to move these files to trash. We choose to move files to trash as opposed to deleting them entirely in case we need to undo this operation later down the line.

In [ ]:
# Fill in the folder and pattern to match
folder_id = "14idmMBbM5xXZg4b61iINHbBTl2z4yLeD"
pattern = "2.txt"

# Call the function
find_and_trash_files(folder_id, pattern)

Trashing file: ATL_10_20_Shary_WWLLN_Locations 2.txt (ID: 19cWRDiv1VkCASM4JqzjPc7T3keV3W9DB)
Trashing file: ATL_10_18_Paula_Reduced_Trackfile 2.txt (ID: 18aFdGtXVjd6vRoHp29tHUsmeTgRQLAj_)
Trashing file: ATL_10_9_Gaston_WWLLN_Locations 2.txt (ID: 1ATJpfVymHpvq2PB4JjuQpnKu9J7i5XaA)
Trashing file: ATL_10_9_Gaston_Reduced_Trackfile 2.txt (ID: 1fXEARdA_8Rt-byFCpgg9x-oBBcX4ON1I)
Trashing file: ATL_10_11_Igor_Reduced_Trackfile 2.txt (ID: 1L9Hai3wnxcje7hjnJyjMo0MJj6l-OuZW)
Trashing file: ATL_10_7_Earl_Reduced_Trackfile 2.txt (ID: 12IAQy4hafTjJzWILRxuoieKWraHgn3lB)
Trashing file: ATL_10_11_Igor_WWLLN_Locations 2.txt (ID: 18HUVwgnT09NOwcWu6_gqxlHbnKK8fJn2)
Trashing file: ATL_10_7_Earl_WWLLN_Locations 2.txt (ID: 11E4pvf5m4AfvJV2eqXT8-KxKwgi-kfqV)
Trashing file: ATL_10_16_Nicole_Reduced_Trackfile 2.txt (ID: 1XKr_Ct7fYS7Thj0ASORZOhuMoPeruxPm)
Trashing file: ATL_10_16_Nicole_WWLLN_Locations 2.txt (ID: 1ttDmUx5aU4xZKseQp3Q_r4F-aupcPQrD)
Trashing file: ATL_10_6_Danielle_Reduced_Trackfile 2.txt (ID: 19